In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [11]:
import os
import pandas as pd

data_path = '../experiments/scenario_age_pred/data/'

source_data = pd.read_csv(os.path.join(data_path, 'transactions_train.csv'))
source_data.head(2)

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017
2,33172,8,11,13.887
3,33172,9,11,15.983
4,33172,10,11,21.341


In [12]:
df_target = pd.read_csv(os.path.join(data_path, 'train_target.csv'))
df_target = df_target.set_index('client_id')

df_target.head(2)

,bins
client_id,
24662,2
1046,0
34089,2
34848,1
47076,3


In [4]:
import logging
logging.basicConfig(level=logging.INFO, format='%(funcName)-20s   : %(message)s')

In [13]:
from dltranz.data_preprocessing import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='client_id',
    cols_event_time='trans_date',
    time_transformation='float',
    cols_category=["trans_date", "small_group"],
    cols_log_norm=["amount_rur"],
    print_dataset_info=False,
)

In [15]:
%%time

dataset = preprocessor.fit_transform(source_data)

_td_float              : To-float time transformation
transform              : Feature collection in progress ...
transform              : Prepared features for 30000 clients


CPU times: user 1min 41s, sys: 11 s, total: 1min 52s
Wall time: 1min 53s


In [16]:
from dltranz.data_preprocessing.util import update_with_target

dataset = update_with_target(features, df_target, 'client_id', 'bins')

In [22]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.8, random_state=42)